In [31]:
START_YEAR = 2005
CURRENT_SEASON = 2024
ONLY_CURRENT_SEASON = False
SPORT_IDS = [1,11,12,13,14,15,16]

Get FangraphsIds From Chadwick Register

In [20]:
FILE_NAME = "https://github.com/chadwickbureau/register/raw/master/data/people-"
FILE_EXT = ".csv"
FILE_NUM = [0,1,2,3,4,5,6,7,8,9,"a","b","c","d","e","f"]

In [21]:
import requests
import sqlite3
import pandas as pd
import io
from tqdm import tqdm

In [22]:
db = sqlite3.connect("BaseballStats.db")

In [30]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")

for num in tqdm(FILE_NUM):
    file = FILE_NAME + str(num) + FILE_EXT
    response = requests.get(file)
    if response.status_code != 200:
        print(f"Failed to Get {file} : {response.status_code}")
        continue
    
    reqData = response.content
    df = pd.read_csv(io.StringIO(reqData.decode('utf-8')), on_bad_lines="skip")
    df = df[df['key_fangraphs'].notna()][df['key_mlbam'].notna()]
    df['key_mlbam'] = df['key_mlbam'].astype('Int64')
    df['key_fangraphs'] = df['key_fangraphs'].astype('Int64')
    for row in df.itertuples():
        cursor.execute(f"UPDATE Player Set fangraphsId='{row.key_fangraphs}' WHERE mlbId='{row.key_mlbam}'")
    
cursor.execute("END TRANSACTION")
db.commit()

  0%|          | 0/16 [00:00<?, ?it/s]C:\Users\nitzr\AppData\Local\Temp\ipykernel_26932\1149116774.py:13: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(reqData.decode('utf-8')), on_bad_lines="skip")
C:\Users\nitzr\AppData\Local\Temp\ipykernel_26932\1149116774.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df['key_fangraphs'].notna()][df['key_mlbam'].notna()]
  6%|▋         | 1/16 [00:00<00:12,  1.23it/s]C:\Users\nitzr\AppData\Local\Temp\ipykernel_26932\1149116774.py:13: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(reqData.decode('utf-8')), on_bad_lines="skip")
C:\Users\nitzr\AppData\Local\Temp\ipykernel_26932\1149116774.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df['key_fangraphs'].notna()][df['key_mlbam'].notna()]
 12%|█▎ 

Get War From Fangraphs

In [32]:
import pybaseball

In [57]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")

for year in tqdm(range(START_YEAR, CURRENT_SEASON)):
    if ONLY_CURRENT_SEASON and year != CURRENT_SEASON:
        continue
    
    # For some reason no OPS breaks it
    hittingStats = pybaseball.batting_stats(year, qual=0, stat_columns=["PA", "BsR", "Off", "Def", "WAR", "OPS"])
    for row in hittingStats.itertuples():
        mlbId = cursor.execute("SELECT mlbId FROM Player WHERE fangraphsId=?", (row.IDfg,)).fetchone()[0]
        if mlbId == None or cursor.execute("SELECT COUNT(*) FROM Player_YearlyWar WHERE mlbId=? AND year=? AND position=?", (mlbId, year, "hitting")).fetchone()[0] > 0:
            continue
        
        cursor.execute("INSERT INTO Player_YearlyWar VALUES(?,?,?,?,?,?,?,?)", (mlbId, year, "hitting", row.PA, row.WAR, row.Off, row.Def, row.BsR))
    
    
    # Similar to above, W is needed
    pitchingStats = pybaseball.pitching_stats(year, qual=0, stat_columns = ["IP", "WAR", "W"])
    for row in pitchingStats.itertuples():
        mlbId = cursor.execute("SELECT mlbId FROM Player WHERE fangraphsId=?", (row.IDfg,)).fetchone()[0]
        if mlbId == None or cursor.execute("SELECT COUNT(*) FROM Player_YearlyWar WHERE mlbId=? AND year=? and position=?", (mlbId, year, "pitching")).fetchone()[0] > 0:
            continue
        
        innings, subinnings = str(row.IP).split('.')
        outs = 3 * innings + subinnings
        
        cursor.execute("INSERT INTO Player_YearlyWar VALUES(?,?,?,?,?,?,?,?)", (mlbId, year, "pitching", outs, row.WAR, 0, 0, 0))
        
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 19/19 [01:30<00:00,  4.78s/it]


Update the status of player careers

In [58]:
UPDATE_YEAR = 2023

In [59]:
NUM_EMPTY_YEARS = 2 # Players who have not played in last N years are considered retired
SERVICE_TIME_CUTOFF = 6 # Players who have reached this service time have reached FA and no longer accrue prospect value
AGED_OUT_AGE = 27 # If a player has not made the MLB by this age, the prospect value becomes 
AGED_OUT_MONTH = 4
ROOKIE_CUTTOFF_DAYS = 60

Insert any players that do not exist

In [61]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
idDatas = cursor.execute("SELECT mlbId, position FROM Player ORDER BY mlbId DESC").fetchall()
for id, position in tqdm(idDatas):
    if cursor.execute("SELECT COUNT(*) FROM Player_CareerStatus WHERE mlbId=? AND position=?", (id, position)).fetchone()[0] > 0:
        continue
    
    cursor.execute("INSERT INTO Player_CareerStatus('mlbId','position') VALUES(?,?)", (id, position))
    
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 53721/53721 [00:00<00:00, 318694.63it/s]


Update primary position for players who don't have one

In [66]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
unknownPositions = cursor.execute("SELECT mlbId, position FROM Player_CareerStatus WHERE isPrimaryPosition IS NULL").fetchall()
for id, position in tqdm(unknownPositions):
    sumBF = cursor.execute("SELECT SUM(battersFaced) FROM Player_Pitcher_MonthStats WHERE mlbId=?", (id,)).fetchone()[0]
    sumPA = cursor.execute("SELECT SUM(AB) FROM Player_Hitter_MonthStats WHERE mlbId=?", (id,)).fetchone()[0]
    
    # If not found at all, need to set to zero
    if sumBF == None:
        sumBF = 0
    if sumPA == None:
        sumPA = 0

    # Set to whichever position has more
    # Only player this doesn't work for is Ohtani, who will be excluded for being a NPB signee
    if (sumBF == 0) and (sumPA == 0):
        continue
    isHitter = sumPA > sumBF
    if (isHitter and position=="hitting") or (not isHitter and position=="pitching"):
        isPrimary = 1
    else:
        isPrimary = 0
        
    cursor.execute("UPDATE Player_CareerStatus SET isPrimaryPosition=? WHERE mlbId=? AND position=?", (isPrimary, id, position))
    
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 53721/53721 [00:01<00:00, 43513.13it/s]


Check if player is currently active

In [69]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
idDatas = cursor.execute("SELECT mlbId, position FROM Player_CareerStatus WHERE isPrimaryPosition='1'").fetchall()
for id, position in tqdm(idDatas):
    if position == "hitting":
        table = "Player_Hitter_MonthStats"
    else:
        table = "Player_Pitcher_MonthStats"
        
    lastYear = cursor.execute(f"SELECT Year FROM {table} WHERE mlbId=? ORDER BY Year DESC LIMIT 1", (id,)).fetchone()[0]
    if (UPDATE_YEAR - lastYear) >= 2:
        isActive = 0
    else:
        isActive = 1
        
    cursor.execute("UPDATE Player_CareerStatus SET isActive=? WHERE mlbId=?", (isActive, id))
    
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 43970/43970 [00:00<00:00, 271283.71it/s]


Check if player has had their service time reached

In [72]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
for id, position in tqdm(idDatas):
    serviceYears = cursor.execute("SELECT serviceYear FROM Player_ServiceTime WHERE mlbId=? ORDER BY serviceYear DESC", (id,)).fetchone()
    if serviceYears == None:
        serviceReached = 0
    elif serviceYears[0] < SERVICE_TIME_CUTOFF:
        serviceReached = 0
    else:
        serviceReached = 1
        
    cursor.execute("UPDATE Player_CareerStatus SET serviceReached=? WHERE mlbId=?", (serviceReached, id))

cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 43970/43970 [00:00<00:00, 283804.07it/s]


Set MLB Start Year

In [115]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
for id, _ in tqdm(idDatas, desc="MLB Start Year Check"):
    pitchingYear = cursor.execute("SELECT year FROM Player_Pitcher_MonthStats WHERE mlbId=? AND level=? ORDER BY year ASC LIMIT 1", (id, 1)).fetchone()
    hittingYear = cursor.execute("SELECT year FROM Player_Hitter_MonthStats WHERE mlbId=? AND levelId=? ORDER BY year ASC LIMIT 1", (id, 1)).fetchone()
    if pitchingYear == None:
        if hittingYear == None: #Never made mlb
            continue
        
        # only made as hitter
        startYear = hittingYear[0]
    else:
        if hittingYear == None: # only made as pitcher
            startYear = pitchingYear[0]
        else: # made as both
            if hittingYear[0] < pitchingYear[0]:
                startYear = hittingYear[0]
            else:
                startYear = pitchingYear[0]
            
    
    #startYear = cursor.execute("SELECT year FROM Player_YearlyWar WHERE mlbId=? ORDER BY year ASC", (id,)).fetchone()
    if startYear is not None:
        cursor.execute("UPDATE Player_CareerStatus SET mlbStartYear=? WHERE mlbId=?", (startYear, id))

cursor.execute("END TRANSACTION")
db.commit()

MLB Start Year Check: 100%|██████████| 43970/43970 [00:00<00:00, 289281.40it/s]


Set Service End Year

In [79]:
db.rollback()
cursor = db.cursor()
mlbIdDatas = cursor.execute("SELECT mlbId, position FROM Player_CareerStatus WHERE mlbStartYear IS NOT NULL").fetchall()
cursor.execute("BEGIN TRANSACTION")
for id, position in tqdm(idDatas):
    serviceData = cursor.execute("SELECT year, serviceYear, serviceDays FROM Player_ServiceTime WHERE mlbId=? ORDER BY year DESC", (id,)).fetchall()
    if serviceData == None or len(serviceData) == 0:
        continue
    
    if serviceData[0][1] < 6: # Service time has not been reached
        continue 
    
    for year, sy, sd in serviceData:
        if sy < SERVICE_TIME_CUTOFF or (sy == SERVICE_TIME_CUTOFF and sd == 0):
            cursor.execute("UPDATE Player_CareerStatus SET serviceEndYear=? WHERE mlbId=?", (year, id))
            break
        
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 43970/43970 [00:00<00:00, 578268.01it/s]


Get first year of career

Need to check if any pre-05 data exists to exclude

In [102]:
db.rollback()
cursor = db.cursor()
file = open("errors/error.txt", "w")

noCareerStartData = cursor.execute("SELECT mlbId FROM Player_CareerStatus WHERE careerStartYear IS NULL ORDER BY mlbId ASC").fetchall()
for i, id in enumerate(tqdm(noCareerStartData)):
    id = id[0]
    response = requests.get(f"https://statsapi.mlb.com/api/v1/people/{id}?hydrate=currentTeam,team,stats(type=[yearByYear](team(league)),leagueListId=mlb_milb)&site=en")
    if response.status_code != 200:
        print(f"Error for id={id}: {response.status_code}")
        continue
    
    jsonData = response.json()
    try:
        year = jsonData["people"][0]["stats"][0]["splits"][0]["season"]
        cursor.execute("UPDATE Player_CareerStatus SET careerStartYear=? WHERE mlbId=?", (year, id))
    except Exception as e:
        file.write(f"{id}\n")
        
    if i % 100 == 0:
        db.commit()
        
file.close()
db.commit()

 50%|█████     | 1/2 [00:00<00:00,  4.39it/s]

Error for id=0: 404


100%|██████████| 2/2 [00:00<00:00,  4.47it/s]

Error for id=529900: 404


Use Selenium to get the players from error.txt to manually populate

In [95]:
from selenium import webdriver

In [103]:
db.rollback()
cursor = db.cursor()

file = open("errors/error.txt", "r")
driver = webdriver.Chrome()

for id in file:
    driver.get(f"https://www.milb.com/player/{id}")
    year = input("Start Year")
    cursor.execute("UPDATE Player_CareerStatus SET careerStartYear=? WHERE mlbId=?", (year, id))
    db.commit()
    cursor = db.cursor()


Check if player did not reach MLB before certain age

First, check players who have not made MLB

In [106]:
db.rollback()
cursor = db.cursor()

noMlbPlayers = cursor.execute("SELECT mlbId FROM Player_CareerStatus WHERE mlbStartYear IS NULL").fetchall()
cursor.execute("BEGIN TRANSACTION")
cutoffYear = CURRENT_SEASON + 1 - AGED_OUT_AGE
for id in tqdm(noMlbPlayers):
    id = id[0]
    birthYear, birthMonth = cursor.execute("SELECT birthYear, birthMonth FROM Player WHERE mlbId=?", (id,)).fetchone()
    
    # Need to eliminate any players who don't have a listed birth date
    if birthYear == None or birthMonth == None:
        cursor.execute("UPDATE Player_CareerStatus SET agedOut=?, ignorePlayer=? WHERE mlbId=?", (1, 1, id))
        continue
    
    if birthMonth >= 4:
        birthYear += 1
    if birthYear < cutoffYear:
        agedOut = 1
    else:
        agedOut = 0
        
    cursor.execute("UPDATE Player_CareerStatus SET agedOut=? WHERE mlbId=?", (agedOut, id))

cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 44603/44603 [00:00<00:00, 237165.83it/s]


Next, check that players who made the MLB didn't have their first year after 27

In [108]:
db.rollback()
cursor = db.cursor()

noMlbPlayers = cursor.execute("SELECT mlbId, mlbStartYear FROM Player_CareerStatus WHERE mlbStartYear IS NOT NULL").fetchall()
cursor.execute("BEGIN TRANSACTION")
for id, startYear in tqdm(noMlbPlayers):
    birthYear, birthMonth = cursor.execute("SELECT birthYear, birthMonth FROM Player WHERE mlbId=?", (id,)).fetchone()
    if birthMonth >= 4:
        birthYear += 1
    
    if startYear - birthYear > AGED_OUT_AGE:
        agedOut = 1
    else:
        agedOut = 0
        
    cursor.execute("UPDATE Player_CareerStatus SET agedOut=? WHERE mlbId=?", (agedOut, id))

cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 9118/9118 [00:00<00:00, 178792.26it/s]


Check to see when a player reached rookie eligibility (60 service days)

After this point, the player is no longer a prospect by the model

In [118]:
db.rollback()
cursor = db.cursor()
mlbIdDatas = cursor.execute("SELECT mlbId FROM Player_CareerStatus WHERE mlbStartYear IS NOT NULL").fetchall()
cursor.execute("BEGIN TRANSACTION")
for id in tqdm(mlbIdDatas):
    id = id[0]
    serviceData = cursor.execute("SELECT year, serviceYear, serviceDays FROM Player_ServiceTime WHERE mlbId=? ORDER BY year ASC", (id,)).fetchall()
    if serviceData == None or len(serviceData) == 0:
        continue
    
    # Check that the last element (first entry) has service time <= 1 year.  Don't have full data
    if serviceData[-1][1] > 1 or (serviceData[-1][1] == 1 and serviceData[-1][2] > 0):
        continue
    
    for year, sy, sd in serviceData:
        if sy > 0 or sd >= SERVICE_TIME_CUTOFF:
            cursor.execute("UPDATE Player_CareerStatus SET mlbRookieYear=? WHERE mlbId=?", (year, id))
            break
        
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 9378/9378 [00:00<00:00, 240461.33it/s]


Last column is ignorePlayer.  This is intended for players that the model doesn't have good data to predict (eg players from Japan/Korea that go directly to the MLB)

The file with the player list will be edited manually

In [119]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
cursor.execute("UPDATE Player_CareerStatus SET ignorePlayer=NULL")

file = open("ignorePlayers.txt", "r")
for id in file:
    cursor.execute("UPDATE Player_CareerStatus SET ignorePlayer=? WHERE mlbId=?", (1, id))

cursor.execute("END TRANSACTION")
db.commit()

Set the highest level the player has reached

This is a seperate table, but should be joined if the db is refactored